<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

In [2]:
# ─── SETUP FOR FAULT CONTROLLERS ───

import os, sys, subprocess

try:
    REPO_DIR = "/content/Cloud_Computing"
    LOGIC_DIR = os.path.join(REPO_DIR, "logic")

    # Step 1: Clone the repo if not present
    if not os.path.isdir(REPO_DIR):
        subprocess.run([
            "git", "clone", "-b", "develop",
            "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
            REPO_DIR
        ], check=True)

    # Step 2: Pull latest changes from develop
    subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

    # Step 3: Add logic folder to sys.path
    sys.path.append(LOGIC_DIR)

    # Step 4: Install dependencies
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 5: Import required notebooks
    from importnb import Notebook
    with Notebook():
        import Fault_controller as fc
        import repair_controller as rc

    print("✅ Fault setup completed successfully.")

except Exception as e:
    print("❌ Fault setup failed:", str(e))

omer_cohen 050-111-2222 200
shira_gold 050-222-3333 200
daniel_bar 050-333-4444 200
roni_benami 050-444-5555 200
lior_levi 050-555-6666 200
yael_elyashiv 050-666-7777 200
noa_katz 050-777-8888 200
matan_tal 050-888-9999 200
or_peled 050-999-0000 200
tom_segal 050-000-1111 200
Daniel Bar | 050-333-4444 | Robotics Engineer
Lior Levi | 050-555-6666 | Backend Developer
Matan Tal | 050-888-9999 | Mechanical Engineer
Noa Katz | 050-777-8888 | UX Designer
Omer Cohen | 050-111-2222 | Automation Engineer
Or Peled | 050-999-0000 | System Engineer
Roni Ben Ami | 050-444-5555 | QA Engineer
Shira Gold | 050-222-3333 | Electrical Engineer
Tom Segal | 050-000-1111 | Production Engineer
Yael Elyashiv | 050-666-7777 | Hardware Engineer
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressure': 963.5, 'Distance': 8.9}
Status: 200 | Uploaded: {'Temperature': 36.5, 'Hu

In [3]:
import gradio as gr
from datetime import datetime

# ---------- Active faults Helpers ----------
def get_severity_color(severity):
    return {
        "Low": "#fff176",
        "Medium": "#ffb74d",
        "High": "#ef5350"
    }.get(severity, "#fff176")

def format_datetime(iso_str):
    dt = datetime.fromisoformat(iso_str)
    return dt.strftime("%-d/%-m, %-I:%M %p")

def split_sensor(sensor):
    parts = sensor.split()
    return " ".join(parts[:2]), " ".join(parts[2:]) if len(parts) > 2 else ""

def update_main_and_table(sort_mode, sensor_order):
    main_fault, rows = active_faults_table_data(sort_mode, sensor_order)
    return gr.update(value=rows), main_fault

def get_main_fault():
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict) or not faults:
        return None, "", "", ""

    first_key = next(iter(faults))  # This is the actual Firebase key
    fault = faults[first_key]

    title_words = fault.get("title", "").split()
    sensor1 = " ".join(title_words[:2])
    sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""

    return fault, sensor1, sensor2, first_key

def get_sort_key(fault, sort_mode, sensor_order):
    severity_rank = {"Low": 1, "Medium": 2, "High": 3}
    sev = severity_rank.get(fault.get("severity", ""), 0)

    if sensor_order == "Indoor → Outdoor":
        sensor_val = 0 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 1
    elif sensor_order == "Outdoor → Indoor":
        sensor_val = 1 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 0
    else:
        sensor_val = 0

    if sort_mode == "Low → High":
        return (sev, sensor_val)
    elif sort_mode == "High → Low":
        return (-sev, sensor_val)
    else:  # By Time
        return (sensor_val,) if sensor_order != "Original Order" else ()

def active_faults_table_data(sort_mode="By Time", sensor_order="Original Order", main_fault_title=None):
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict) or not faults:
        return None, []

    fault_items = list(faults.items())

    # Exclude the current main fault
    if main_fault_title:
        fault_items = [item for item in fault_items if item[0] != main_fault_title]

    if sort_mode != "By Time" or sensor_order != "Original Order":
        fault_items = sorted(
            fault_items,
            key=lambda x: get_sort_key(x[1], sort_mode, sensor_order)
        )

    rows = []
    for _, fault in fault_items:
        iso_ts = fault.get("timestamp", "")
        try:
            dt = datetime.fromisoformat(iso_ts)
            time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
        except Exception:
            time_str = iso_ts[:16].replace("T", " ")

        rows.append([
            time_str,
            fault.get("title", ""),
            fault.get("severity", ""),
            fault.get("status", "")
        ])

    return None, rows

def update_repair_box(main_key):
    faults = fc.fb.get_active_faults()
    fault = faults.get(main_key) if faults else None

    if not fault:
        return (
            gr.update(value="No Active Faults To Repair"),  # XP text
            gr.update(value=""),                            # sensor name
            gr.update(visible=False),                       # start_btn
            gr.update(choices=[], value=[], interactive=False),  # checklist
            gr.update(visible=False),                       # finish_btn
        )

    severity_to_xp = {"Low": 50, "Medium": 100, "High": 200}
    xp = severity_to_xp.get(fault.get("severity", "Low"), 50)

    title = fault.get("title", "")
    title_words = title.split()
    sensor_snippet = " ".join(title_words[:2]) if len(title_words) >= 2 else title

    return (
        gr.update(value=f"Earn {xp} XP by fixing"),                             # XP text
        gr.update(value=f"{sensor_snippet} Sensor Whithin 10 Minutes"),                # sensor text
        gr.update(visible=True),                                                # start_btn
        gr.update(choices=fault["actions"], value=[], interactive=False),       # checklist
        gr.update(visible=False),                                               # finish_btn (still hidden initially)
    )

def resolved_faults_table_data():
    faults = fc.fb.get_resolved_faults()
    if not isinstance(faults, dict) or not faults:
        return []

    rows = []
    for _, fault in faults.items():
        try:
            time_str = format_datetime(fault.get("timestamp", ""))
        except:
            time_str = fault.get("timestamp", "")[:16].replace("T", " ")

        rows.append([
            time_str,
            fault.get("sensor", ""),
            fault.get("severity", ""),
            fault.get("repaired_by", ""),
            fault.get("status", "Resolved")
        ])
    return rows

# ----------- Resolved faults Helper -----------
def resolved_faults_table_data():
    try:
        faults = fc.fb.get_resolved_faults()
        if not isinstance(faults, dict) or not faults:
            return []

        rows = []
        for _, fault in faults.items():
            try:
                time_str = format_datetime(fault.get("timestamp", ""))
            except:
                time_str = fault.get("timestamp", "")[:16].replace("T", " ")

            rows.append([
                time_str,
                fault.get("sensor", ""),
                fault.get("severity", ""),
                fault.get("repaired_by", ""),
                fault.get("status", "Resolved")
            ])
        return rows
    except Exception as e:
        print("❌ resolved_faults_table_data() error:", e)
        return []

# ---------- Extracted ----------
presented_fault, sensor1, sensor2, presented_key = get_main_fault()
severity_color = get_severity_color(presented_fault["severity"])
title_words = presented_fault.get("title", "").split()
sensor1 = " ".join(title_words[:2])
sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""
time_str = format_datetime(presented_fault["timestamp"])
xp_value = {"Low": 50, "Medium": 100, "High": 200}.get(presented_fault["severity"], 50)

# ---------- Styles ----------
css = f"""
#main-fault {{
    background-color: {severity_color};
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#main-fault * {{
    color: white !important;
}}
#main-fault ul {{
  padding-left: 15px !important;
  margin-top: 4px;
  margin-bottom: 0;
}}

#repair-box {{
    background-color: white;
    color: black;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#repair-box * {{
    color: black !important;
}}
.repair-checkbox {{
  display: flex !important;
  flex-direction: row !important;
  gap: 20px;
  flex-wrap: wrap;
}}
"""

# ---------- App ----------
with gr.Blocks(css=css) as demo:

    # -------- First Row ----------
    with gr.Row():
        with gr.Column(elem_id="main-fault", scale=1):
            gr.Markdown("### FAULT DETECTED")
            gr.Markdown(f"**{time_str}**")
            gr.Markdown(f"### {sensor1}")
            gr.Markdown(sensor2)
            gr.Markdown("#### Suggested Actions:")
            for action in presented_fault["actions"]:
                gr.Markdown(f"- {action}")

        with gr.Column(elem_id="repair-box", scale=1):
            gr.Markdown("## REPAIR CHALLENGE")
            repair_xp_text = gr.Markdown("")
            repair_sensor_text = gr.Markdown("")
            gr.Markdown("#### Repair Steps")

            repair_checklist = gr.CheckboxGroup(
                choices=[],
                value=[],
                interactive=False,
                label="",
                elem_classes=["repair-checklist"]
            )

            start_btn = gr.Button("Start Repair", visible=True)
            finish_btn = gr.Button("Finish Repair", visible=False, interactive=False)
            status_output = gr.Textbox(label="System Message", interactive=False)

    # -------- Second Row ----------
    with gr.Row():
        with gr.Column():
            with gr.Row():
                gr.Markdown("### Active Faults")

                sensor_filter = gr.Dropdown(
                    choices=["Original Order", "Indoor → Outdoor", "Outdoor → Indoor"],
                    value="Original Order",  # default
                    label="",
                    interactive=True,
                    scale=1,
                    elem_classes=["small-dropdown"]
                )
                sort_choice = gr.Dropdown(
                    choices=["By Time", "Low → High", "High → Low"],
                    value="By Time",      # default
                    label="",
                    interactive=True,
                    scale=1,
                    elem_classes=["small-dropdown"]
                )


            table_headers = gr.Dataframe(
                headers=["Time", "Sensor", "Severity", "Status"],
                value=[],
                row_count=0,
                col_count=(4, "fixed"),
                interactive=False,
                label=""
        )
    # -------- Third Row ----------
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Resolved Faults")
            resolved_table = gr.Dataframe(
                headers=["Time Detected", "Sensor", "Severity", "Repaired by", "Status"],
                value=[],                     # Start empty
                row_count=0,
                col_count=(5, "fixed"),
                interactive=False,
                label=""
            )

    # ---------- Logic ----------
    def start_real_repair(main_key):
        faults = fc.fb.get_active_faults()                                      # Get the active fault dictionary from Firebase
        fault = faults.get(main_key)                                            # Get the specific fault by key

        if not fault or not isinstance(fault, dict):
            return (
                gr.update(choices=[], value=[], interactive=False),             # checklist
                gr.update(visible=True),                                        # start_btn
                gr.update(visible=False)                                        # finish_btn
            )

        # Make sure actions are a list
        actions = fault.get("actions")
        # Start the repair (write to Firebase)
        msg = rc.start_repair(main_key)

        return (
            gr.update(choices=actions, value=[], interactive=True),             # checklist
            gr.update(visible=False),                                           # start_btn
            gr.update(visible=True),                                            # finish_btn
            msg                                                                 # status_output
        )

    def handle_checkbox_update(selected, main_key):
        faults = fc.fb.get_active_faults()
        fault = faults[main_key]
        expected = fault.get("actions", [])
        return gr.update(interactive=(selected == expected))


    def complete_real_repair(main_key):
        msg = rc.complete_repair(main_key)  # Again, use the Firebase key
        return (
            gr.update(visible=False),
            gr.update(value="Start Repair", visible=True, interactive=True),
            msg
        )


    def load_active_faults():
        rows = active_faults_table_data()
        return gr.update(value=rows)


    # ---------- Bindings ----------
    presented_key_state = gr.State(presented_key)

    start_btn.click(
        fn=start_real_repair,
        inputs=[presented_key_state],
        outputs=[repair_checklist, start_btn, finish_btn, status_output]
    )

    finish_btn.click(
        fn=complete_real_repair,
        inputs=[presented_key_state],
        outputs=[finish_btn, start_btn, status_output]
    )

    repair_checklist.change(
        fn=handle_checkbox_update,
        inputs=[repair_checklist, presented_key_state],
        outputs=finish_btn
    )

    sort_choice.change(
        fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
        inputs=[sort_choice, sensor_filter, presented_key_state],
        outputs=[table_headers]
    )

    sensor_filter.change(
        fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
        inputs=[sort_choice, sensor_filter, presented_key_state],
        outputs=[table_headers]
    )

    demo.load(
        fn=lambda key: (
            *update_repair_box(key),
            active_faults_table_data("By Time", "Original Order", key)[1],  # 6th
            resolved_faults_table_data()
        ),
        inputs=[presented_key_state],
        outputs=[
            repair_xp_text,
            repair_sensor_text,
            start_btn,
            repair_checklist,
            finish_btn,
            table_headers,
            resolved_table
        ]
    )



demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2e00f42f3d40244bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
